In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
mydir = 'data_dir/raw_data/'
os.listdir(mydir)

['holidays_events.csv',
 'oil.csv',
 'preprocessed_train_data.csv',
 'sample_submission.csv',
 'stores.csv',
 'test.csv',
 'train.csv',
 'transactions.csv']

In [3]:
train = pd.read_csv(mydir + 'train.csv')
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [4]:
x_train = train.drop(columns = ['sales'])
x_train.tail()

,id,date,store_nbr,family,onpromotion
3000883,3000883,2017-08-15,9,POULTRY,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,1
3000885,3000885,2017-08-15,9,PRODUCE,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,8
3000887,3000887,2017-08-15,9,SEAFOOD,0


In [5]:
x_test = pd.read_csv(mydir + 'test.csv')
x_test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [6]:
train_data = pd.concat([x_train,x_test])
train_data.head()

,id,date,store_nbr,family,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0
1,1,2013-01-01,1,BABY CARE,0
2,2,2013-01-01,1,BEAUTY,0
3,3,2013-01-01,1,BEVERAGES,0
4,4,2013-01-01,1,BOOKS,0


In [7]:
stores_df   = pd.read_csv(mydir+'stores.csv')    
stores_df   = stores_df.drop_duplicates()
stores_df   = stores_df.dropna()
stores_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [8]:
train_data = pd.merge(train_data, stores_df, on = 'store_nbr', how= 'left')

In [9]:
city_state_map = stores_df[['state','city']].drop_duplicates()
city_state_map.sort_values('state')

,state,city
36,Azuay,Cuenca
18,Bolivar,Guaranda
13,Chimborazo,Riobamba
11,Cotopaxi,Latacunga
39,El Oro,Machala
42,Esmeraldas,Esmeraldas
34,Guayas,Playas
35,Guayas,Libertad
23,Guayas,Guayaquil
26,Guayas,Daule


In [10]:
he_df   = pd.read_csv(mydir+'holidays_events.csv')   
he_df   = he_df[he_df['transferred']==False] #elimination the transferred holidays
he_df['city']  = he_df.apply(lambda row: row['locale_name'] if row['locale'].lower() == 'local' else 'NA', axis=1)
he_df['state'] = he_df.apply(lambda row: row['locale_name'] if row['locale'].lower() == 'regional' else 'NA', axis=1)
he_df['state_mapped'] = pd.merge(he_df['city'],city_state_map,on = 'city', how = 'left')['state']
he_df['state'] = he_df.apply(lambda row: row['state_mapped'] if row['state'].lower() == 'na' else row['state'], axis=1)
he_df = he_df.drop(columns = ['state_mapped'])
he_df = he_df.rename(columns = {'type':'holiday_type'})
he_df.head()

,date,holiday_type,locale,locale_name,description,transferred,city,state
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,Manta,Manabi
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,NA,Cotopaxi
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,Cuenca,Azuay
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,Libertad,Guayas
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,Riobamba,Chimborazo


In [11]:
#Divide the he_df with types of locale = national

he_df_national  = he_df.loc[he_df['locale'] == 'National',['date','holiday_type']]
he_df_national  = he_df_national.rename(columns = {'holiday_type' : 'holiday_type_national' })
he_df_national['IS_NATIONAL_HOLIDAY'] = 1
he_df_national

,date,holiday_type_national,IS_NATIONAL_HOLIDAY
14,2012-08-10,Holiday,1
20,2012-10-12,Transfer,1
21,2012-11-02,Holiday,1
22,2012-11-03,Holiday,1
31,2012-12-21,Additional,1
...,...,...,...
345,2017-12-22,Additional,1
346,2017-12-23,Additional,1
347,2017-12-24,Additional,1
348,2017-12-25,Holiday,1


In [12]:
train_data = pd.merge(train_data, he_df_national, on = 'date', how= 'left')

In [13]:
#Divide the he_df with types of locale = regional

he_df_regional  = he_df.loc[he_df['locale'] == 'Regional',['date','state','holiday_type']]
he_df_regional  = he_df_regional.rename(columns = {'holiday_type' : 'holiday_type_state' })
he_df_regional['IS_REGIONAL_HOLIDAY'] = 1
he_df_regional

,date,state,holiday_type_state,IS_REGIONAL_HOLIDAY
1,2012-04-01,Cotopaxi,Holiday,1
7,2012-06-25,Imbabura,Holiday,1
23,2012-11-06,Santo Domingo de los Tsachilas,Holiday,1
24,2012-11-07,Santa Elena,Holiday,1
47,2013-04-01,Cotopaxi,Holiday,1
58,2013-06-25,Imbabura,Holiday,1
76,2013-11-06,Santo Domingo de los Tsachilas,Holiday,1
77,2013-11-07,Santa Elena,Holiday,1
96,2014-04-01,Cotopaxi,Holiday,1
112,2014-06-25,Imbabura,Holiday,1


In [14]:
train_data = pd.merge(train_data, he_df_regional, on = ['date','state'], how= 'left')

In [15]:
#Divide the he_df with types of locale = local

he_df_local  = he_df.loc[he_df['locale'] == 'Local',['date','city','holiday_type']]
he_df_local  = he_df_local.rename(columns = {'holiday_type' : 'holiday_type_city' })
he_df_local['IS_LOCAL_HOLIDAY'] = 1
he_df_local

,date,city,holiday_type_city,IS_LOCAL_HOLIDAY
0,2012-03-02,Manta,Holiday,1
2,2012-04-12,Cuenca,Holiday,1
3,2012-04-14,Libertad,Holiday,1
4,2012-04-21,Riobamba,Holiday,1
5,2012-05-12,Puyo,Holiday,1
...,...,...,...,...
338,2017-11-12,Ambato,Holiday,1
339,2017-12-05,Quito,Additional,1
341,2017-12-08,Loja,Holiday,1
342,2017-12-08,Quito,Transfer,1


In [16]:
train_data = pd.merge(train_data, he_df_local, on = ['date','city'], how= 'left')


In [17]:
train_data['holiday_type'] = train_data['holiday_type_city'].fillna(train_data['holiday_type_state']).fillna(train_data['holiday_type_national'])
train_data=train_data.drop(columns = ['holiday_type_city','holiday_type_state','holiday_type_national'])

train_data.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster,IS_NATIONAL_HOLIDAY,IS_REGIONAL_HOLIDAY,IS_LOCAL_HOLIDAY,holiday_type
0,0,2013-01-01,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,1.0,NaN,NaN,Holiday
1,1,2013-01-01,1,BABY CARE,0,Quito,Pichincha,D,13,1.0,NaN,NaN,Holiday
2,2,2013-01-01,1,BEAUTY,0,Quito,Pichincha,D,13,1.0,NaN,NaN,Holiday
3,3,2013-01-01,1,BEVERAGES,0,Quito,Pichincha,D,13,1.0,NaN,NaN,Holiday
4,4,2013-01-01,1,BOOKS,0,Quito,Pichincha,D,13,1.0,NaN,NaN,Holiday


In [18]:
train_data['IS_NATIONAL_HOLIDAY'] = train_data['IS_NATIONAL_HOLIDAY'].fillna(0)
train_data['IS_REGIONAL_HOLIDAY'] = train_data['IS_REGIONAL_HOLIDAY'].fillna(0)
train_data['IS_LOCAL_HOLIDAY'] = train_data['IS_LOCAL_HOLIDAY'].fillna(0)
train_data['holiday_type'] = train_data['holiday_type'].fillna('None')

train_data['IS_HOLIDAY'] = train_data['IS_LOCAL_HOLIDAY'] + train_data['IS_REGIONAL_HOLIDAY'] + train_data['IS_NATIONAL_HOLIDAY']
train_data['IS_HOLIDAY'] = np.clip(train_data['IS_HOLIDAY'], 0, 1)
#train_data  = train_data.drop(columns = ['IS_NATIONAL_HOLIDAY','IS_REGIONAL_HOLIDAY','IS_LOCAL_HOLIDAY'])

In [19]:
oil_df  = pd.read_csv(mydir +'oil.csv')
oil_df  = oil_df.drop_duplicates()
oil_df  = oil_df.dropna(subset = 'dcoilwtico')
oil_df.head()

,date,dcoilwtico
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21


In [20]:
train_data = pd.merge(train_data, oil_df, on = 'date', how= 'left')

In [21]:
# Calculate the mean target value for each category in the categorical columns
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()


train_data['city'] = le.fit_transform(train_data['city'])
train_data['state'] = le.fit_transform(train_data['state'])
train_data['type'] = le.fit_transform(train_data['type'])
train_data['holiday_type'] = le.fit_transform(train_data['holiday_type'])

#train_data['city'] = train_data.groupby('city')['sales'].transform('mean')
#train_data['state'] = train_data.groupby('state')['sales'].transform('mean')
#train_data['type'] = train_data.groupby('type')['sales'].transform('mean')
#train_data['holiday_type'] = train_data.groupby('holiday_type')['sales'].transform('mean')

#train_data = train_data.drop(columns = ['city','state','type','holiday_type','IS_NATIONAL_HOLIDAY','IS_REGIONAL_HOLIDAY','IS_LOCAL_HOLIDAY'])
train_data = train_data.drop(columns = ['IS_NATIONAL_HOLIDAY','IS_REGIONAL_HOLIDAY','IS_LOCAL_HOLIDAY'])

In [22]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

train_data['family'] = le.fit_transform(train_data['family'])


In [23]:
# Convert the 'onpromotion' column to 1 if positive, else 0
train_data['onpromotion'] = train_data['onpromotion'].apply(lambda x: 1 if x > 0 else 0)


In [24]:
#train_data_v2 = train_data.drop(columns = ['sales','transactions']) #dropping transaction data
train = train_data.rename(columns = {'dcoilwtico':'oil_price'})
train['oil_price'] = train['oil_price'].fillna(method='bfill').fillna(method='ffill')

train.tail()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster,holiday_type,IS_HOLIDAY,oil_price
3036787,3029395,2017-08-31,9,28,1,18,12,1,6,4,0.0,47.26
3036788,3029396,2017-08-31,9,29,0,18,12,1,6,4,0.0,47.26
3036789,3029397,2017-08-31,9,30,1,18,12,1,6,4,0.0,47.26
3036790,3029398,2017-08-31,9,31,1,18,12,1,6,4,0.0,47.26
3036791,3029399,2017-08-31,9,32,0,18,12,1,6,4,0.0,47.26


In [25]:
# Convert 'date' to datetime format
train['date'] = pd.to_datetime(train['date'])

# Extract month and year
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train['day'] = train['date'].dt.day
# Extract week of the day (Sunday=0, Monday=1, ..., Saturday=6)
train['day_of_week'] = (train['date'].dt.weekday + 1) % 7

# Extract week of the month (1 to 7 = w1, 8 to 15 = w2, 16 to 23 = w3, 24 to end = w4)
train['week_of_month'] = (train['date'].dt.day - 1) // 7 + 1

train['week_number'] = train['date'].dt.isocalendar().week
#train['sales'] = pd.to_numeric(train['sales'], errors='coerce')
#train.dropna(subset=['sales'], inplace=True)


In [26]:
# Keep the first occurrence of each 'id'
train = train.drop_duplicates(subset=['id'], keep='first').reset_index(drop=True)

In [27]:
train.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster,holiday_type,IS_HOLIDAY,oil_price,month,year,day,day_of_week,week_of_month,week_number
0,0,2013-01-01,1,0,0,18,12,3,13,3,1.0,93.14,1,2013,1,2,1,1
1,1,2013-01-01,1,1,0,18,12,3,13,3,1.0,93.14,1,2013,1,2,1,1
2,2,2013-01-01,1,2,0,18,12,3,13,3,1.0,93.14,1,2013,1,2,1,1
3,3,2013-01-01,1,3,0,18,12,3,13,3,1.0,93.14,1,2013,1,2,1,1
4,4,2013-01-01,1,4,0,18,12,3,13,3,1.0,93.14,1,2013,1,2,1,1


In [28]:
test = train[train['date'] > '2017-08-01']
train = train[train['date'] <= '2017-08-15']
train.shape

(3000888, 18)

In [29]:
y_train = pd.read_csv(mydir + 'train.csv')
y_train = y_train[['id','sales']]
y_train = y_train.drop_duplicates()
#y_train = y_train.drop_duplicates()


In [30]:
train_all = pd.merge(train,y_train,on='id', how='left')
train_all.columns

Index(['id', 'date', 'store_nbr', 'family', 'onpromotion', 'city', 'state',
       'type', 'cluster', 'holiday_type', 'IS_HOLIDAY', 'oil_price', 'month',
       'year', 'day', 'day_of_week', 'week_of_month', 'week_number', 'sales'],
      dtype='object')

In [31]:
train_all.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster,holiday_type,IS_HOLIDAY,oil_price,month,year,day,day_of_week,week_of_month,week_number,sales
0,0,2013-01-01,1,0,0,18,12,3,13,3,1.0,93.14,1,2013,1,2,1,1,0.0
1,1,2013-01-01,1,1,0,18,12,3,13,3,1.0,93.14,1,2013,1,2,1,1,0.0
2,2,2013-01-01,1,2,0,18,12,3,13,3,1.0,93.14,1,2013,1,2,1,1,0.0
3,3,2013-01-01,1,3,0,18,12,3,13,3,1.0,93.14,1,2013,1,2,1,1,0.0
4,4,2013-01-01,1,4,0,18,12,3,13,3,1.0,93.14,1,2013,1,2,1,1,0.0


In [33]:
test.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster,holiday_type,IS_HOLIDAY,oil_price,month,year,day,day_of_week,week_of_month,week_number
2975940,2975940,2017-08-02,1,0,0,18,12,3,13,4,0.0,49.6,8,2017,2,3,1,31
2975941,2975941,2017-08-02,1,1,0,18,12,3,13,4,0.0,49.6,8,2017,2,3,1,31
2975942,2975942,2017-08-02,1,2,1,18,12,3,13,4,0.0,49.6,8,2017,2,3,1,31
2975943,2975943,2017-08-02,1,3,1,18,12,3,13,4,0.0,49.6,8,2017,2,3,1,31
2975944,2975944,2017-08-02,1,4,0,18,12,3,13,4,0.0,49.6,8,2017,2,3,1,31


In [32]:
train_all.to_csv('data_dir/wip_data/preprocessed_train_data_v10.csv')
test.to_csv('data_dir/wip_data/preprocessed_test_data_v10.csv')